In [1]:
"""
OANDA Live Trading Bot - Michael Harris FVG Strategy
Configurable timeframe and instrument
"""

from config import access_token, accountID
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import MarketOrderRequest, TakeProfitDetails, StopLossDetails
import oandapyV20.endpoints.orders as orders
from oandapyV20 import API
from apscheduler.schedulers.blocking import BlockingScheduler
import pytz
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:

# ========================================
# CONFIGURATION
# ========================================
INSTRUMENT = "EUR_USD"  # Change to: BTC_USD, ETH_USD, etc.
# Options: Gran.M1, Gran.M5, Gran.M15, Gran.M30, Gran.H1, Gran.H4, etc.
TIMEFRAME = Gran.M15
TRADE_SIZE = 1000       # Units to trade
SL_PERCENTAGE = 0.04    # 4% Stop Loss
TP_PERCENTAGE = 0.02    # 2% Take Profit
USE_LIVE = False        # Set True for live trading, False for practice

# Timeframe to cron schedule mapping
TIMEFRAME_CRON = {
    Gran.M1: {'minute': '*'},
    Gran.M5: {'minute': '*/5'},
    Gran.M15: {'minute': '1,16,31,46'},
    Gran.M30: {'minute': '1,31'},
    Gran.H1: {'minute': '1'},
    Gran.H4: {'minute': '1', 'hour': '*/4'},
}

In [3]:
# ========================================
# MICHAEL HARRIS SIGNAL DETECTION
# ========================================

def detect_harris_signal(df):
    """
    Detects Michael Harris FVG pattern signals
    Returns: 2 for BUY, 1 for SELL, 0 for no signal
    """
    if len(df) < 4:
        return 0

    current_pos = len(df) - 1

    # Buy condition (Bullish FVG Pattern)
    c1 = df['High'].iloc[current_pos] > df['High'].iloc[current_pos-1]
    c2 = df['High'].iloc[current_pos-1] > df['Low'].iloc[current_pos]
    c3 = df['Low'].iloc[current_pos] > df['High'].iloc[current_pos-2]
    c4 = df['High'].iloc[current_pos-2] > df['Low'].iloc[current_pos-1]
    c5 = df['Low'].iloc[current_pos-1] > df['High'].iloc[current_pos-3]
    c6 = df['High'].iloc[current_pos-3] > df['Low'].iloc[current_pos-2]
    c7 = df['Low'].iloc[current_pos-2] > df['Low'].iloc[current_pos-3]

    if c1 and c2 and c3 and c4 and c5 and c6 and c7:
        return 2  # BUY signal

    # Sell condition (Bearish FVG Pattern)
    c1 = df['Low'].iloc[current_pos] < df['Low'].iloc[current_pos-1]
    c2 = df['Low'].iloc[current_pos-1] < df['High'].iloc[current_pos]
    c3 = df['High'].iloc[current_pos] < df['Low'].iloc[current_pos-2]
    c4 = df['Low'].iloc[current_pos-2] < df['High'].iloc[current_pos-1]
    c5 = df['High'].iloc[current_pos-1] < df['Low'].iloc[current_pos-3]
    c6 = df['Low'].iloc[current_pos-3] < df['High'].iloc[current_pos-2]
    c7 = df['High'].iloc[current_pos-2] < df['High'].iloc[current_pos-3]

    if c1 and c2 and c3 and c4 and c5 and c6 and c7:
        return 1  # SELL signal

    return 0  # No signal

In [4]:

# ========================================
# FETCH LIVE CANDLES
# ========================================

def fetch_candles(instrument_pair, granularity, n=200):
    """
    Fetch candles from OANDA
    """
    client = CandleClient(access_token, real=USE_LIVE)
    collector = client.get_collector(instrument_pair, granularity)
    candles = collector.grab(n)

    df = pd.DataFrame([{
        'Open': float(str(c.bid.o)),
        'High': float(str(c.bid.h)),
        'Low': float(str(c.bid.l)),
        'Close': float(str(c.bid.c)),
    } for c in candles])

    df.reset_index(drop=True, inplace=True)
    return df

In [5]:

# ========================================
# TRADING JOB
# ========================================


def trading_job():
    print(f"\n{'='*60}")
    print(f"[{datetime.now(pytz.timezone('America/Chicago'))}]")
    print(f"Running Michael Harris Strategy on {INSTRUMENT} ({TIMEFRAME})")
    print(f"{'='*60}")

    try:
        # Convert instrument string to Pair enum
        instrument_pair = getattr(Pair, INSTRUMENT)

        # Fetch candles
        df = fetch_candles(instrument_pair, TIMEFRAME, n=200)

        # Detect signal
        signal = detect_harris_signal(df)

        current_price = df['Close'].iloc[-1]
        print(f"Current Price: {current_price:.5f}")
        print(f"Signal: {signal} (0=None, 1=SELL, 2=BUY)")

        if signal == 0:
            print("No valid trade signal at this time.")
            return

        # Calculate SL and TP based on strategy parameters
        if signal == 2:  # BUY
            sl = current_price - (SL_PERCENTAGE * current_price)
            tp = current_price + (TP_PERCENTAGE * current_price)
            units = TRADE_SIZE

            print(f"\n🟢 BUY SIGNAL DETECTED")
            print(f"Entry: {current_price:.5f}")
            print(f"Stop Loss: {sl:.5f} ({SL_PERCENTAGE*100}% below entry)")
            print(f"Take Profit: {tp:.5f} ({TP_PERCENTAGE*100}% above entry)")
            print(f"Risk/Reward: 1:{TP_PERCENTAGE/SL_PERCENTAGE:.2f}")

        elif signal == 1:  # SELL
            sl = current_price + (SL_PERCENTAGE * current_price)
            tp = current_price - (TP_PERCENTAGE * current_price)
            units = -TRADE_SIZE

            print(f"\n🔴 SELL SIGNAL DETECTED")
            print(f"Entry: {current_price:.5f}")
            print(f"Stop Loss: {sl:.5f} ({SL_PERCENTAGE*100}% above entry)")
            print(f"Take Profit: {tp:.5f} ({TP_PERCENTAGE*100}% below entry)")
            print(f"Risk/Reward: 1:{TP_PERCENTAGE/SL_PERCENTAGE:.2f}")

        # Validate SL/TP
        if signal == 2 and (tp <= current_price or sl >= current_price):
            print("❌ Invalid SL/TP for BUY order.")
            return
        if signal == 1 and (tp >= current_price or sl <= current_price):
            print("❌ Invalid SL/TP for SELL order.")
            return

        # Create market order
        mo = MarketOrderRequest(
            instrument=INSTRUMENT,
            units=units,
            takeProfitOnFill=TakeProfitDetails(price=f"{tp:.5f}").data,
            stopLossOnFill=StopLossDetails(price=f"{sl:.5f}").data
        )

        # Execute order
        client = API(access_token)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)

        print(f"\n✅ Order executed successfully!")
        print(
            f"Order ID: {rv.get('orderFillTransaction', {}).get('id', 'N/A')}")
        print(
            f"Fill Price: {rv.get('orderFillTransaction', {}).get('price', 'N/A')}")

    except AttributeError:
        print(
            f"❌ Error: Invalid instrument '{INSTRUMENT}'. Check Pair enum in oanda_candles.")
    except Exception as e:
        print(f"❌ Order failed: {str(e)}")

In [6]:

# ========================================
# SCHEDULER
# ========================================

if __name__ == "__main__":
    print("="*60)
    print("OANDA MICHAEL HARRIS FVG STRATEGY BOT")
    print("="*60)
    print(f"Instrument: {INSTRUMENT}")
    print(f"Timeframe: {TIMEFRAME}")
    print(f"Trade Size: {TRADE_SIZE} units")
    print(f"Stop Loss: {SL_PERCENTAGE*100}%")
    print(f"Take Profit: {TP_PERCENTAGE*100}%")
    print(f"Mode: {'LIVE' if USE_LIVE else 'PRACTICE'}")
    print("="*60)

    scheduler = BlockingScheduler()

    # Get cron settings for the selected timeframe
    cron_settings = TIMEFRAME_CRON.get(TIMEFRAME, {'minute': '*/15'})

    scheduler.add_job(
        trading_job,
        'cron',
        day_of_week='mon-fri',
        hour=cron_settings.get('hour', '0-23'),
        minute=cron_settings.get('minute', '*/15'),
        timezone='America/Chicago'
    )

    print(f"\n🤖 Bot started. Waiting for next {TIMEFRAME} candle close...")
    print("Press Ctrl+C to stop.\n")

    try:
        scheduler.start()
    except (KeyboardInterrupt, SystemExit):
        print("\n🛑 Bot stopped by user.")

OANDA MICHAEL HARRIS FVG STRATEGY BOT
Instrument: EUR_USD
Timeframe: M15
Trade Size: 1000 units
Stop Loss: 4.0%
Take Profit: 2.0%
Mode: PRACTICE

🤖 Bot started. Waiting for next M15 candle close...
Press Ctrl+C to stop.


[2025-11-11 03:01:00.053897-06:00]
Running Michael Harris Strategy on EUR_USD (M15)
Current Price: 1.15538
Signal: 0 (0=None, 1=SELL, 2=BUY)
No valid trade signal at this time.

[2025-11-11 03:16:00.041783-06:00]
Running Michael Harris Strategy on EUR_USD (M15)
Current Price: 1.15543
Signal: 0 (0=None, 1=SELL, 2=BUY)
No valid trade signal at this time.

[2025-11-11 03:31:00.020065-06:00]
Running Michael Harris Strategy on EUR_USD (M15)
Current Price: 1.15636
Signal: 0 (0=None, 1=SELL, 2=BUY)
No valid trade signal at this time.

[2025-11-11 03:46:00.034547-06:00]
Running Michael Harris Strategy on EUR_USD (M15)
Current Price: 1.15658
Signal: 0 (0=None, 1=SELL, 2=BUY)
No valid trade signal at this time.

[2025-11-11 04:01:00.017876-06:00]
Running Michael Harris Strategy